In [1]:
!pip install openpyxl



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd

In [3]:
# Показать все листы в Excel
df = pd.read_excel("data/Товары для мэтчинга.xlsx", sheet_name=None)
print(df.keys())

dict_keys(['База товаров', 'Товары для проверки', 'Соответсвия товаров'])


In [4]:
df = pd.read_excel("data/Товары для мэтчинга.xlsx", sheet_name="База товаров")

In [5]:
df_test = pd.read_excel("data/Товары для мэтчинга.xlsx", sheet_name="Соответсвия товаров")

In [6]:
del(df_test["Наименование"])

In [7]:
df_test["test_name"] = df_test["Unnamed: 2"]
del(df_test["Unnamed: 2"])

In [8]:
del(df_test["Unnamed: 3"])    

In [9]:
del(df_test["Unnamed: 4"])    

In [10]:
df.head()

,Артикул,Наименование
0,Т0000017607,"Абрикос, кг"
1,102058684,"Авокадо, вес"
2,Т0000051246,"Апельсины, кг"
3,Т0000012233,"Бананы Эквадор, кг"
4,101988505,Бедро цыпленка-бройлера охл. вес.


In [12]:
df.to_csv("data/products.tsv", index=False, sep="\t", encoding="utf-8-sig")

In [13]:
df_test.dropna(subset=["test_name"], inplace=True)

In [14]:
# df_test вывести все не текстовые колонки
print(df_test.select_dtypes(exclude=['object']).columns)

Index([], dtype='object')


In [15]:
# df_test пустые колонки
print(df_test.isnull().sum())
# вывести их
print(df_test[df_test.isnull().any(axis=1)])


Артикул      0
test_name    0
dtype: int64
Empty DataFrame
Columns: [Артикул, test_name]
Index: []


In [16]:
df_test.dtypes

Артикул      object
test_name    object
dtype: object

In [40]:
!pip install fuzzywuzzy


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [41]:
import fuzzywuzzy
import rapidfuzz
import logging
import time


In [42]:
class Timer:
    def __enter__(self):
        self.start = time.time()
        return self  # возвращаем self, чтобы получить доступ к elapsed_time

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time.time()
        self.elapsed = round(self.end - self.start,4)
        #print(f"Cell execution time: {self.elapsed:.4f} seconds")


In [48]:

import fuzzywuzzy.process


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class BaseMatcher:
    def __init__(self, df, threshold=80):
        self.df = df
        self.threshold = threshold

    def find_best_match(self, ocr_name):
        raise NotImplementedError("This method should be overridden by subclasses")

class FuzzyWuzzySimpleMatcher(BaseMatcher):
    def __init__(self, df, threshold=80, **kwargs):
        self.df = df
        self.threshold = threshold
        self.kwargs = kwargs

    def find_best_match(self, ocr_name):
        choices = self.df['Наименование'].tolist()
        result = fuzzywuzzy.process.extractOne(ocr_name, choices,  score_cutoff = self.threshold, **self.kwargs)
        if result is None:
            logging.warning(f"No match found for: {ocr_name}")
            return None

        match, score = result
        idx = choices.index(match)

        logging.info(f"Match: {match}, Score: {score}")
        return self.df.iloc[idx],score

class FuzzyRapidSimpleMatcher(BaseMatcher):
    def __init__(self, df, threshold=80, **kwargs):
        self.df = df
        self.threshold = threshold
        self.kwargs = kwargs
    def find_best_match(self, ocr_name):
        choices = self.df['Наименование'].tolist()
        result = rapidfuzz.process.extractOne(ocr_name, choices, score_cutoff=self.threshold, **self.kwargs)

        if result is None:
            logging.warning(f"No match found for: {ocr_name}")
            return None

        match, score, idx = result
        logging.info(f"Match: {match}, Score: {score}")
        return self.df.iloc[idx],score


class FuzzyRapidByWordsMatcher(BaseMatcher):
    def __init__(self, df, threshold=80, **kwargs):
        self.df = df
        self.threshold = threshold
        self.kwargs = kwargs
    
    def find_best_match(self, ocr_name):
        words = ocr_name.split()
        best_match = None
        best_score = 0
        for index, row in self.df.iterrows():
            name = row['Наименование']
            score = rapidfuzz.fuzz.token_set_ratio(ocr_name, name)
            if score > best_score and score >= self.threshold:
                best_score = score
                best_match = row

        logging.info(f"Best match for {ocr_name}: {best_match['Наименование']} with score {best_score}")
        if best_score >= self.threshold:
            return best_match
        return None

def match_products(test_df,  matcher):
    matcher 
    matches = []
    
    for index, row in test_df.iterrows():
        ocr_name = row['test_name']
        with Timer() as t:
            match_result = matcher.find_best_match(ocr_name)
        
        if match_result is not None:
            match, score = match_result
            matched = False
            if match["Артикул"] == row['Артикул']:
                matched = True
                logging.info(f"Match found for {ocr_name}: {match['Наименование']} with Article {match['Артикул']}")
            else:
                logging.warning(f"Match found for {ocr_name} but article does not match: {match['Артикул']} vs {row['Артикул']}")
            
            matches.append({
                'Test Name': ocr_name,
                'Matched Name': match['Наименование'],
                'Article': match['Артикул'],
                "Ожидаемый Артикул": row['Артикул'],
                "matched": matched,
                "score": score,
                "elapsed_time": t.elapsed
            })
          
        else:
            matched = False
            matches.append({
                'Test Name': ocr_name,
                'Matched Name': None,
                'Article': None,
                "Ожидаемый Артикул": row['Артикул'],
                "matched": matched,
                "score": None,
                "elapsed_time": t.elapsed
            })
            logging.warning(f"No match found for {ocr_name}")

    return pd.DataFrame(matches)

In [49]:


fuzzywuzzy_scorers = [
    fuzzywuzzy.fuzz.ratio,    
    fuzzywuzzy.fuzz.token_sort_ratio,
    fuzzywuzzy.fuzz.token_set_ratio,

    fuzzywuzzy.fuzz.partial_ratio,
    fuzzywuzzy.fuzz.partial_token_sort_ratio,
    fuzzywuzzy.fuzz.partial_token_set_ratio,

    fuzzywuzzy.fuzz.WRatio,
    fuzzywuzzy.fuzz.QRatio,
#    rapidfuzz.fuzz.partial_ratio_alignment # Будет ошибка
    
]

In [50]:
matcher = FuzzyWuzzySimpleMatcher(df, threshold=20)
match = matcher.find_best_match("Пельмени")
print(f"Match: {match},")

INFO:root:Match: Бульмени со свининой и говядиной п/п 700г., Score: 68


Match: (Артикул                                          102718021
Наименование    Бульмени со свининой и говядиной п/п 700г.
Name: 6, dtype: object, 68),


In [51]:
logging.disable(logging.CRITICAL + 1)

for scorer  in fuzzywuzzy_scorers:
    print(f"Using scorer: {scorer.__name__}")
    matcher = FuzzyWuzzySimpleMatcher(df, threshold=40, scorer=scorer)
    #match = matcher.find_best_match("Пельмени")
    #print(f"Match: {match},")
    
    result_df = match_products(df_test, matcher)
    # result_df посчитать accuracy по колонке matched
    accuracy = result_df['matched'].mean() * 100
    print(f"Accuracy: {accuracy:.2f}%")
    result_df.to_excel(f"data/Результаты мэтчинга_fuzzwuzzy_{scorer.__name__}_{accuracy:.2f}.xlsx", index=False)


Using scorer: ratio
Accuracy: 78.67%
Using scorer: token_sort_ratio
Accuracy: 74.67%
Using scorer: token_set_ratio
Accuracy: 89.33%
Using scorer: partial_ratio
Accuracy: 68.00%
Using scorer: partial_token_sort_ratio
Accuracy: 73.33%
Using scorer: partial_token_set_ratio
Accuracy: 28.00%
Using scorer: WRatio
Accuracy: 49.33%
Using scorer: QRatio
Accuracy: 78.67%


In [227]:
rapidfuzz_scorers = [
    rapidfuzz.fuzz.ratio,    
    rapidfuzz.fuzz.token_sort_ratio,
    rapidfuzz.fuzz.token_set_ratio,

    rapidfuzz.fuzz.partial_ratio,
    rapidfuzz.fuzz.partial_token_sort_ratio,
    rapidfuzz.fuzz.partial_token_set_ratio,

    rapidfuzz.fuzz.WRatio,
    rapidfuzz.fuzz.QRatio,
#    rapidfuzz.fuzz.partial_ratio_alignment # Будет ошибка
    
]

In [ ]:
logging.disable(logging.CRITICAL + 1)

for scorer in rapidfuzz_scorers:
    print(f"Using scorer: {scorer.__name__}")
    matcher = FuzzyRapidSimpleMatcher(df, threshold=40, scorer=scorer)
    #match = matcher.find_best_match("Пельмени")
    #print(f"Match: {match},")
    
    result_df = match_products(df_test, matcher)
    # result_df посчитать accuracy по колонке matched
    accuracy = result_df['matched'].mean() * 100
    print(f"Accuracy: {accuracy:.2f}%")
    result_df.to_excel(f"data/Результаты мэтчинга_rapidfuzz_{scorer.__name__}_{accuracy:.2f}.xlsx", index=False)


Using scorer: QRatio
Accuracy: 69.33%
Using scorer: QRatio
Accuracy: 62.67%
Using scorer: QRatio
Accuracy: 72.00%
Using scorer: QRatio
Accuracy: 65.33%
Using scorer: QRatio
Accuracy: 69.33%
Using scorer: QRatio
Accuracy: 42.67%
Using scorer: QRatio
Accuracy: 58.67%
Using scorer: QRatio
Accuracy: 69.33%
